In [7]:
import numpy as np
import nltk
# nltk.download('punkt')
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

#preprocessing
def tokenize(sentence):
    return nltk.word_tokenize(sentence)


def stem(word):
    return stemmer.stem(word.lower())


def bag_of_words(tokenized_sentence, words):
    # stem each word
    sentence_words = [stem(word) for word in tokenized_sentence]
    # initialize bag with 0 for each word
    bag = np.zeros(len(words), dtype=np.float32)
    for idx, w in enumerate(words):
        if w in sentence_words: 
            bag[idx] = 1

    return bag

In [8]:
import numpy as np
import random
import json

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

#from nltk_utils import bag_of_words, tokenize, stem
#from model import NeuralNet

with open('intents.json', 'r') as f:
    intents = json.load(f)

all_words = []
tags = []
xy = []
# loop through each sentence in our intents patterns
for intent in intents['intents']:
    tag = intent['tag']
    # add to tag list
    tags.append(tag)
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = tokenize(pattern)
        # add to our words list
        all_words.extend(w)
        # add to xy pair
        xy.append((w, tag)) #mapping b/w i/p & o/p in training

# stem and lower each word
ignore_words = ['?', '.', '!']
all_words = [stem(w) for w in all_words if w not in ignore_words]
# remove duplicates and sort
all_words = sorted(set(all_words))
tags = sorted(set(tags))

print(len(xy), "patterns")
print(len(tags), "tags:", tags)
print(len(all_words), "unique stemmed words:", all_words)

# create training data
X_train = []
y_train = []
for (pattern_sentence, tag) in xy:
    # X: bag of words for each pattern_sentence
    bag = bag_of_words(pattern_sentence, all_words)
    X_train.append(bag)
    # y: PyTorch CrossEntropyLoss needs only class labels, not one-hot
    label = tags.index(tag)
    y_train.append(label)

X_train = np.array(X_train)
y_train = np.array(y_train)

class ChatDataset(Dataset):

    def __init__(self):
        self.n_samples = len(X_train)  #length of the sample
        self.x_data = X_train          #Feature patterns stored in x_data
        self.y_data = y_train          # The output tagsstored in y_data

    # support indexing such that dataset[i] can be used to get i-th sample
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    # we can call len(dataset) to return the size
    def __len__(self):
        return self.n_samples
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    #from sklearn import tree
    #model = tree.DecisionTreeClassifier()
    #model.fit(X_train,y_train)

94 patterns
16 tags: ['Cold_drinks', 'French fries', 'chicken_momo', 'corn_momo', 'delivery', 'goodbye', 'greeting', 'items', 'menu_list', 'paneer_momo', 'payments', 'price', 'thanks', 'timing', 'veg_momo', 'wb']
100 unique stemmed words: ["'s", 'a', 'accept', 'allow', 'anyon', 'are', 'avail', 'buy', 'bye', 'can', 'card', 'cash', 'chicken', 'close', 'cod', 'cold', 'corn', 'cost', 'credit', 'day', 'deliveri', 'do', 'doe', 'drink', 'eat', 'food', 'french', 'fri', 'get', 'give', 'good', 'goodby', 'great', 'have', 'hello', 'help', 'hey', 'hi', 'how', 'i', 'in', 'is', 'it', 'item', 'kind', 'later', 'list', 'long', 'look', 'lot', 'mastercard', 'me', 'menu', 'momo', 'morn', 'much', 'my', 'na', 'need', 'now', 'of', 'ok', 'okay', 'on', 'onli', 'open', 'order', 'paneer', 'pay', 'payment', 'paypal', 'price', 'procedur', 'remain', 'restur', 'see', 'sell', 'serv', 'ship', 'show', 'take', 'thank', 'that', 'the', 'there', 'time', 'to', 'varieti', 'veg', 'wan', 'want', 'what', 'when', 'which', 'will',

In [9]:
import numpy as np

class NeuralNetwork():
    
    def __init__(self):
        # Seed the random number generator
        np.random.seed(1)

        # Set synaptic weights to a 3x1 matrix,
        # with values from -1 to 1 and mean 0
        self.synaptic_weights = 2 * np.random.random((100, 1)) - 1

    def sigmoid(self, x):
        """
        Takes in weighted sum of the inputs and normalizes
        them through between 0 and 1 through a sigmoid function
        """
        return 1 / (1 + np.exp(-x))

    def sigmoid_derivative(self, x):
        """
        The derivative of the sigmoid function used to
        calculate necessary weight adjustments
        """
        return x * (1 - x)

    def train(self, training_inputs, training_outputs, training_iterations):
        """
        We train the model through trial and error, adjusting the
        synaptic weights each time to get a better result
        """
        for iteration in range(training_iterations):
            # Pass training set through the neural network
            output = self.think(training_inputs)

            # Calculate the error rate
            error = training_outputs - output

            # Multiply error by input and gradient of the sigmoid function
            # Less confident weights are adjusted more through the nature of the function
            adjustments = np.dot(training_inputs.T, error * self.sigmoid_derivative(output))

            # Adjust synaptic weights
            self.synaptic_weights += adjustments

    def think(self, inputs):
        """
        Pass inputs through the neural network to get output
        """
        
        inputs = inputs.astype(float)
        output = self.sigmoid(np.dot(inputs, self.synaptic_weights))
        #output = self.relu = nn.ReLU()
        return output


if __name__ == "__main__":

    # Initialize the single neuron neural network
    neural_network = NeuralNetwork()

    print("Random starting synaptic weights: ")
    #print(neural_network.synaptic_weights)

    # The training set, with 4 examples consisting of 3
    # input values and 1 output value
    training_inputs = np.array(X_train)
    
    
    training_outputs = np.array([y_train])
    print(type(training_inputs))
    print(training_inputs.shape)
    #print(training_outputs.shape)
    to = np.transpose(training_outputs)
    print(to.shape)
    to20 = to/20

    # Train the neural network
    neural_network.train(training_inputs, to20, 10000)

    print("Synaptic weights after training: ")
    print(neural_network.synaptic_weights)

Random starting synaptic weights: 
<class 'numpy.ndarray'>
(94, 100)
(94, 1)
Synaptic weights after training: 
[[ 2.67273397e-03]
 [ 5.90297928e-01]
 [ 3.56469117e-01]
 [ 2.69525080e-01]
 [ 5.49144829e-01]
 [-6.73573151e-01]
 [ 3.10339075e+00]
 [ 2.43742720e-01]
 [-1.09861229e+00]
 [-5.64008661e-01]
 [-3.75870724e-01]
 [ 7.49240079e-01]
 [-2.70850434e+00]
 [ 1.81717734e+00]
 [-1.26529031e+00]
 [-3.79495477e+00]
 [-2.23108485e+00]
 [ 2.18626002e-01]
 [ 5.32081639e-01]
 [-2.21547410e-01]
 [-6.13167033e-01]
 [-6.19364752e-01]
 [ 5.48654936e-01]
 [-3.75124456e+00]
 [ 2.43742720e-01]
 [-1.56551164e-01]
 [-2.23881161e+00]
 [-2.33079046e+00]
 [-6.55757979e-01]
 [ 8.51281829e-01]
 [-6.25750450e-01]
 [-1.09861229e+00]
 [ 4.05465108e-01]
 [ 4.56819824e-01]
 [-8.47297860e-01]
 [-1.12924267e-01]
 [-8.47297860e-01]
 [-8.47297860e-01]
 [-1.03601615e-01]
 [ 1.28130005e-01]
 [ 2.43887863e-01]
 [-1.73646827e+00]
 [ 6.21830454e-01]
 [-7.63027235e-01]
 [-1.16532881e+00]
 [ 1.86481777e-01]
 [-7.20824458e-

In [12]:
bot_name = "Oreo"
print("Let's chat! (type 'done' to exit)")
while True:
    sentence = input("You: ")
    if sentence == "done":
        print("Thank you! Visit Again.")
        break

    sentence = tokenize(sentence)
    print(sentence)
    X = bag_of_words(sentence, all_words)
    print(X)
    X = X.reshape(1, X.shape[0])
    print(X)
    xx = neural_network.think(np.array(X))
    print(tags[round(xx[0][0]*20)])
    print(round(xx[0][0]*20))
    #X = torch.from_numpy(X).to(device)
    #print(X)
    
    
    
    #probs = torch.softmax(output, dim=1)#http://localhost:8888/notebooks/TT%20Chatbot%20Oreo.ipynb#
    #prob = probs[0][predicted.item()]
    #if prob.item() > 0.75:
    for intent in intents['intents']:
        if tags[round(xx[0][0]*20)] == intent["tag"]:
           # print(prob.item())
            print(f"{bot_name}: {random.choice(intent['responses'])}")

Let's chat! (type 'done' to exit)
You: Hi
['Hi']
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]]
greeting
6
Oreo: Hi there, what can I do for you?
You: Thanks
['Thanks']
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0